### sklearn.tree.DecisionTreeClassifier
### sklearn.tree.DecisionTreeRegressor
- Root Node : 뿌리 마디로, 시작 노드, 레이블(y)
- Intermediate Node : 중간 마디로, 분류되는 과정
- Terminal Node : 끝 마디로, 최종적으로 분류되는 마지막 기준

#### 주요 Hyperparameter
- max_depth : 최대 가지치기 수
- max_leaf_node : 리프 노드의 최대 개수
- min_sample_leaf : 리프 노드가 되기 위한 최소 샘플수
- 현실적으로 min_sample_leaf만을 가늠할 수 있고, 다른 HP는 결정하기 어려움

##### DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, ccp_alpha=0.0)
##### DecisionTreeRegressor(*, criterion='mse', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, ccp_alpha=0.0)

# 분석 코드 - Classification

In [1]:
# 라이브러리 및 데이터 로드
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import randint
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error

df = pd.read_csv('../input/big-data-certification-study/breast-cancer-wisconsin.csv', encoding='utf-8')
df.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [2]:
# 데이터 분리
X=df.drop(columns=['code','Class'])
y=df[['Class']]
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y,random_state=42)

In [3]:
# 정규화
scaler=MinMaxScaler()
scaler.fit(X_train)
mm_X_train=scaler.transform(X_train)
mm_X_test=scaler.transform(X_test)

In [4]:
# 모델 적용
model=DecisionTreeClassifier()
model.fit(mm_X_train, y_train)
pred_train=model.predict(mm_X_train)
model.score(mm_X_train,y_train)

# 과대적합

1.0

In [5]:
# 혼동행렬, 분류예측 보고서
cm_train=confusion_matrix(y_train,pred_train)
cfr_train=classification_report(y_train,pred_train)
print('혼동행렬 :\n',cm_train,
      '\n\n\n분류예측 보고서 :\n',cfr_train)

혼동행렬 :
 [[333   0]
 [  0 179]] 


분류예측 보고서 :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       333
           1       1.00      1.00      1.00       179

    accuracy                           1.00       512
   macro avg       1.00      1.00      1.00       512
weighted avg       1.00      1.00      1.00       512



In [6]:
# 모델 적용
pred_test=model.predict(mm_X_test)
model.score(mm_X_test, y_test)

0.9473684210526315

In [7]:
# 혼동행렬, 분류예측 보고서
cm_test=confusion_matrix(y_test,pred_test)
cfr_test=classification_report(y_test,pred_test)
print('혼동행렬 :\n',cm_test,'\n\n\n분류예측 보고서 :\n',cfr_test)

혼동행렬 :
 [[105   6]
 [  3  57]] 


분류예측 보고서 :
               precision    recall  f1-score   support

           0       0.97      0.95      0.96       111
           1       0.90      0.95      0.93        60

    accuracy                           0.95       171
   macro avg       0.94      0.95      0.94       171
weighted avg       0.95      0.95      0.95       171



In [8]:
# Hyperparameter Tuning
# Grid Search
param_g = {'max_depth':range(2, 20, 2), 'min_samples_leaf':range(1,50,2)}
grid= GridSearchCV(DecisionTreeClassifier(), param_g, cv=5, return_train_score=True)
grid.fit(mm_X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(2, 20, 2),
                         'min_samples_leaf': range(1, 50, 2)},
             return_train_score=True)

In [9]:
print('Best Parameter :', grid.best_params_)
print('Best Score :',round(grid.best_score_,4))
print('Test Score :',round(grid.score(mm_X_test, y_test),4))

Best Parameter : {'max_depth': 8, 'min_samples_leaf': 1}
Best Score : 0.9628
Test Score : 0.9649


In [10]:
# Randomized Search
param_r={'max_depth':randint(low=1, high=20), 'min_samples_leaf':randint(low=1,high=50)}
random=RandomizedSearchCV(DecisionTreeClassifier(), param_distributions=param_r, cv=5, n_iter=20, return_train_score=True)
random.fit(mm_X_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=20,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7f779f25d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7f779f2ad0>},
                   return_train_score=True)

In [11]:
print('Best Parameter :', random.best_params_) 
print('Best Score :',round(random.best_score_,4)) 
print('Test Score :',round(random.score(mm_X_test, y_test),4))

Best Parameter : {'max_depth': 10, 'min_samples_leaf': 1}
Best Score : 0.955
Test Score : 0.9591


# 분석 코드 - Regression

In [12]:
df2=pd.read_csv('../input/big-data-certification-study/house_price.csv', encoding='utf-8') 
df2.head()

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000


In [13]:
# 데이터 분리 
X=df2.drop(columns=['house_value']) 
y=df2[['house_value']] 
X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=42)

In [14]:
# 정규화 
scale=MinMaxScaler() 
scale.fit(X_train) 
ms_X_train=scale.transform(X_train) 
ms_x_test=scale.transform(X_test)

In [15]:
# 모델 적용 
model_r=DecisionTreeRegressor() 
model_r.fit(ms_X_train, y_train) 
pred_x=model_r.predict(ms_X_train) 
model_r.score(ms_X_train, y_train)

# 과대적합

1.0

In [16]:
pred_y=model_r.predict(ms_x_test) 
model_r.score(ms_x_test,y_test)

0.28131309519610537

In [17]:
# RMSE 
rmse_train=np.sqrt(mean_squared_error(y_train,pred_x)) 
rmse_test=np.sqrt(mean_squared_error(y_test,pred_y)) 
print('Train RMSE :', round(rmse_train), '\nTest  RMSE :', round(rmse_test))

Train RMSE : 0 
Test  RMSE : 81046


In [18]:
# Hyperparameter Tuning 
# Grid Search 
g_param = {'max_depth':range(2, 20, 2), 'min_samples_leaf':range(1,50,2)} 
g_search= GridSearchCV(DecisionTreeRegressor(), g_param, cv=5, return_train_score=True) 
g_search.fit(ms_X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': range(2, 20, 2),
                         'min_samples_leaf': range(1, 50, 2)},
             return_train_score=True)

In [19]:
print('Best Parameter :', g_search.best_params_) 
print('Best Score :',round(g_search.best_score_,4)) 
print('Test Score :',round(g_search.score(ms_x_test, y_test),4))

Best Parameter : {'max_depth': 10, 'min_samples_leaf': 39}
Best Score : 0.5847
Test Score : 0.5994


In [20]:
# Randomized Search 
r_param={'max_depth':randint(low=1, high=20), 'min_samples_leaf':randint(low=1,high=50)}
r_search=RandomizedSearchCV(DecisionTreeRegressor(), param_distributions=r_param, cv=5, n_iter=20, return_train_score=True) 
r_search.fit(ms_X_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_iter=20,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7f779f2f10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7f779f2a50>},
                   return_train_score=True)

In [21]:
print('Best Parameter :', r_search.best_params_) 
print('Best Score :',round(r_search.best_score_,4)) 
print('Test Score :',round(r_search.score(ms_x_test, y_test),4))

Best Parameter : {'max_depth': 17, 'min_samples_leaf': 44}
Best Score : 0.5846
Test Score : 0.5979
